In [0]:
employee_data = [
    (1, "Alice", "IT", 60000, "2021-01-10", "India"),
    (2, "Bob", "HR", 45000, "2020-03-15", None),
    (3, "Charlie", "IT", 80000, "2019-07-23", "USA"),
    (4, "David", "Finance", 55000, "2021-11-01", "India"),
    (5, "Eva", "HR", None, "2022-02-19", "UK"),
    (6, "Frank", "IT", 60000, "2021-01-10", "India"),
    (6, "Frank", "IT", 60000, "2021-01-10", "India")
]

emp_cols = ["emp_id", "name", "dept", "salary", "join_date", "country"]


emp_df = spark.createDataFrame(employee_data, emp_cols)
emp_df.show()


In [0]:
dept_data = [
    ("IT", "Technology"),
    ("HR", "Human Resources"),
    ("Finance", "Finance & Accounting")
]

dept_cols = ["dept", "dept_name"]

dept_df = spark.createDataFrame(dept_data, dept_cols)
dept_df.show()


In [0]:
emp_df = emp_df.dropDuplicates(["emp_id"])


In [0]:
from pyspark.sql.functions import col, when

emp_df = emp_df.withColumn(
    "salary",
    when(col("salary").isNull(), 0).otherwise(col("salary"))
).withColumn(
    "country",
    when(col("country").isNull(), "Unknown").otherwise(col("country"))
)


In [0]:
emp_df = emp_df.filter(col("salary") > 50000)


In [0]:
from pyspark.sql.functions import to_date

emp_df = emp_df.withColumn(
    "join_date",
    to_date(col("join_date"), "yyyy-MM-dd")
)


In [0]:
from pyspark.sql.functions import avg, count

dept_stats = emp_df.groupBy("dept").agg(
    avg("salary").alias("avg_salary"),
    count("emp_id").alias("emp_count")
)

dept_stats.show()


In [0]:
emp_df = emp_df.orderBy(col("salary").desc())


In [0]:
final_df = emp_df.join(dept_df, on="dept", how="left")
final_df.show()


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

window_spec = Window.partitionBy("dept").orderBy(col("salary").desc())

final_df = final_df.withColumn(
    "dept_salary_rank",
    rank().over(window_spec)
)


In [0]:
final_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("employee_final_delta")


In [0]:
spark.sql("SELECT * FROM employee_final_delta").show()


In [0]:
%sql
SELECT * FROM catalog.schema.table;
